# Sistema Inteligente de Recuperación y Generación con LangChain

### Alison Geraldine Valderrama Munar

Implementación de un sistema **RAG (Retrieval-Augmented Generation)** que integra modelos de lenguaje avanzados con búsqueda semántica vectorial para consultas contextuales sobre contenido web técnico.

**Tecnologías:** LangChain, OpenAI GPT-4o, Pinecone Vector DB, BeautifulSoup4

---

## Configuración Inicial

### Librerías Requeridas

Instalación de componentes esenciales:
- **openai, python-dotenv**: Integración con OpenAI y manejo de configuración
- **langchain**: Orquestación de aplicaciones con LLMs
- **langchain-openai, langchain-pinecone**: Conectores especializados
- **bs4**: Extracción de contenido HTML

In [17]:
%pip install -q langchain langchain-text-splitters langchain-community bs4
%pip install -qU langchain-openai
%pip install -qU langchain-pinecone
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Variables de Entorno

Importación de credenciales desde `.env`:
- `OPENAI_API_KEY`: Acceso a GPT-4o y generación de embeddings
- `PINECONE_API_KEY`: Autenticación en base de datos vectorial
- `PINECONE_INDEX_NAME`: Nombre del índice destino
- `LANGCHAIN_API_KEY`: Monitoreo con LangSmith (opcional)
- `LANGCHAIN_TRACING`: Habilitación de trazas de ejecución

In [1]:
import os
from dotenv import load_dotenv

# Importar variables de entorno
load_dotenv(override=True)

# Verificación de claves obligatorias
mandatory_vars = ["OPENAI_API_KEY", "PINECONE_API_KEY", "PINECONE_INDEX_NAME"]
for var in mandatory_vars:
    if not os.getenv(var):
        raise ValueError(f"Variable {var} no está definida en .env")

print("Entorno configurado correctamente.")
print(f"Claves cargadas: {', '.join(mandatory_vars)}")

Entorno configurado correctamente.
Claves cargadas: OPENAI_API_KEY, PINECONE_API_KEY, PINECONE_INDEX_NAME


### Modelos de Inteligencia Artificial

Inicialización de componentes IA:
- **GPT-4o**: Motor de generación de respuestas conversacionales
- **text-embedding-3-small**: Generador de representaciones vectoriales (512 dimensiones) para búsqueda semántica

In [2]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

# Instanciar modelo conversacional
chat_model = init_chat_model("gpt-4o", model_provider="openai")

# Instanciar generador de embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1024)

print("Componentes IA listos.")
print(f"LLM: GPT-4o")
print(f"Embeddings: text-embedding-3-large (1024d)")

Componentes IA listos.
LLM: GPT-4o
Embeddings: text-embedding-3-large (1024d)


### Conexión a Pinecone

Establecer conexión con el servicio de vectores:
- Autenticación mediante API key
- Acceso al índice preconfigurado (arep-taller)
- Verificación de características del índice (512 dims, similitud coseno)
- Creación del vector store para operaciones RAG

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# Conectar con Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
idx_name = os.getenv("PINECONE_INDEX_NAME")
pc_index = pc.Index(idx_name)

# Consultar metadatos del índice
index_stats = pc_index.describe_index_stats()
print(f"Índice: {idx_name}")
print(f"Dimensiones: {index_stats.get('dimension', 'N/A')}")
print(f"Vectores actuales: {index_stats.get('total_vector_count', 0)}")

# Configurar almacén vectorial
vector_db = PineconeVectorStore(embedding=embeddings, index=pc_index, )

print("Conexión a Pinecone establecida.")

## 1. Fase de Indexación: Captura y Preparación de Datos

### 1.1 Descarga de Contenido Web

Obtención de documentación mediante `WebBaseLoader`:
- **Origen**: Artículo técnico sobre agentes de IA
- **Extractor**: BeautifulSoup filtra título, headers y texto principal
- Generación de documento unificado con el contenido completo

In [4]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Definir filtros de extracción HTML
html_filter = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": html_filter},
)
docs = loader.load()

assert len(docs) == 1
print(f"Contenido web cargado")
print(f"Tamaño: {len(docs[0].page_content):,} caracteres")
print(f"URL: {docs[0].metadata.get('source', 'N/A')}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Contenido web cargado
Tamaño: 43,047 caracteres
URL: https://lilianweng.github.io/posts/2023-06-23-agent/


### 1.2 Previsualización del Documento

Muestra de los primeros 500 caracteres para verificar la calidad de la extracción.

In [5]:
print("Fragmento inicial del contenido:")
print("=" * 80)
print(docs[0].page_content[:500])
print("=" * 80)

Fragmento inicial del contenido:


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


### 1.3 Segmentación del Texto

División del documento con `RecursiveCharacterTextSplitter`:
- **Chunk size**: 1000 caracteres por segmento
- **Overlap**: 200 caracteres de solapamiento contextual
- **add_start_index**: Mantener referencia a posición original
- Mejora la precisión en la recuperación de información

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Crear divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
chunks = text_splitter.split_documents(docs)

print(f"Documento dividido en {len(chunks)} segmentos.")
print(f"Longitud promedio: {sum(len(c.page_content) for c in chunks) // len(chunks)} chars")

Documento dividido en 63 segmentos.
Longitud promedio: 718 chars


### 1.4 Almacenamiento Vectorial

Proceso de indexación:
- Conversión de segmentos a vectores 1024D
- Almacenamiento en Pinecone con metadata
- Asignación de IDs únicos por fragmento

In [7]:
doc_ids = vector_db.add_documents(documents=chunks)

print(f"Indexados {len(doc_ids)} fragmentos en Pinecone")
print(f"Muestra de IDs: {doc_ids[:3]}")
print(f"Base de datos actualizada")

Indexados 63 fragmentos en Pinecone
Muestra de IDs: ['c877389d-aec8-42cd-96b3-86c3360ec59e', 'bf57df7d-671e-443f-b22d-e508412049ae', '22de2f0c-d07b-449b-8ce2-b9e0e4bda79f']
Base de datos actualizada


## 2. Motor RAG: Recuperación Contextual y Generación

### 2.1 Tool de Búsqueda Contextual

Función para recuperar información relevante:
- Ejecuta búsqueda por similitud semántica
- Obtiene los 2 documentos más cercanos (k=2)
- Formatea resultados con metadata
- Retorna contenido y objetos documento mediante `@tool`

In [12]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Busca y retorna contexto relevante para responder la pregunta del usuario."""
    results = vector_db.similarity_search(query, k=2)
    context_str = "\n\n".join(
        (f"Metadata: {d.metadata}\nTexto: {d.page_content}")
        for d in results
    )
    return context_str, results

print("Tool de búsqueda creado")
print("Parámetros: k=2 documentos por query")

Tool de búsqueda creado
Parámetros: k=2 documentos por query


### 2.2 Ensamblaje del Agente RAG

Construcción del agente con `create_agent`:
- Vinculación de GPT-4o con tool de búsqueda
- Prompt de sistema para uso estratégico del contexto
- Arquitectura que combina recuperación y generación
- Soporte para búsquedas múltiples en queries complejas

In [13]:
from langchain.agents import create_agent

tools_list = [retrieve_context]

# Prompt del sistema
sys_prompt = (
    "Cuentas con una herramienta que recupera contexto de artículos técnicos. "
    "Úsala de forma inteligente para dar respuestas bien fundamentadas. "
    "Puedes hacer varias búsquedas si la pregunta lo requiere."
)

# Crear agente RAG
agent = create_agent(chat_model, tools_list, system_prompt=sys_prompt)

print("Agente RAG operacional")
print("Stack: GPT-4o + Pinecone Search")

Agente RAG operacional
Stack: GPT-4o + Pinecone Search


## 3. Prueba del Sistema

Evaluación con query de múltiples pasos:
- Busca información sobre "Task Decomposition"
- Investiga extensiones del método encontrado
- Streaming de respuesta en tiempo real
- Trazabilidad del razonamiento del agente

In [14]:
multi_step_query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

print(f"Query Multi-paso: {multi_step_query}\n")
print("="*80)
print("Streaming activado...\n")

for event in agent.stream(
    {"messages": [{"role": "user", "content": multi_step_query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

Query Multi-paso: What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.

Streaming activado...

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_eeU52UMAQPhdfqfCgZ9FVGuL)
 Call ID: call_eeU52UMAQPhdfqfCgZ9FVGuL
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Metadata: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578.0}
Texto: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "

## 4. Test de Validación

Verificación con pregunta directa para evaluar respuestas básicas del sistema.

In [15]:
simple_query = "What is task decomposition?"

print(f"Query Directa: {simple_query}\n")
print("="*80)

for event in agent.stream(
    {"messages": [{"role": "user", "content": simple_query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

Query Directa: What is task decomposition?

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_3q01DOy2E3d88ODtnHOUuKV9)
 Call ID: call_3q01DOy2E3d88ODtnHOUuKV9
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

Metadata: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578.0}
Texto: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes

## 5. Inspección del Índice

Revisión del estado del vector store post-indexación.

In [16]:
# Consultar estadísticas actuales
final_stats = pc_index.describe_index_stats()

print("Estado del Vector Store:")
print("="*80)
print(f"     Índice: {idx_name}")
print(f"     Dims: {final_stats.get('dimension', 'N/A')}")
print(f"     Vectores: {final_stats.get('total_vector_count', 0):,}")
print(f"     Similitud: cosine")
print(f"     Infraestructura: serverless (AWS us-east-1)")
# Test de búsqueda
probe_query = "AI agents"
probe_results = vector_db.similarity_search(probe_query, k=1)
print(f"\nTest de búsqueda OK")
print(f"Query: '{probe_query}'")
print(f"Matches: {len(probe_results)}")
if probe_results:
    print(f"Preview: {probe_results[0].page_content[:150]}...")

Estado del Vector Store:
     Índice: arep
     Dims: 1024
     Vectores: 126
     Similitud: cosine
     Infraestructura: serverless (AWS us-east-1)

Test de búsqueda OK
Query: 'AI agents'
Matches: 1
Preview: They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agen...
